In [ ]:
import pandas as pd,sys, numpy as np
from datetime import datetime,timedelta
from google.colab import  drive
drive.mount('/content/drive')
df_temp = pd.read_excel((r'/content/drive/MyDrive/Project 2 intro to DL DFs/NYC temp.xlsx'))
df_temp['date'] = df_temp['date'].astype(str)
df_temp['date'] = pd.Series(map(lambda x: datetime.strptime(str(x),'%Y-%m-%d'),df_temp['date'])).dt.date
print(df_temp.info())
print(df_temp.head(5))

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    31 non-null     object 
 1   tavg    28 non-null     float64
 2   tmin    28 non-null     float64
 3   tmax    28 non-null     float64
 4   prcp    0 non-null      float64
 5   snow    0 non-null      float64
 6   wdir    1 non-null      float64
 7   wspd    27 non-null     float64
 8   wpgt    0 non-null      float64
 9   pres    27 non-null     float64
 10  tsun    0 non-null      float64
dtypes: float64(10), object(1)
memory usage: 2.8+ KB
None
         date  tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt    pres  tsun
0  2020-01-01   3.6   1.7   5.0   NaN   NaN   NaN  17.3   NaN  1008.2   NaN
1  2020-01-02   4.7   0.6   8.9   NaN   NaN   NaN  12.4   NaN  1013.9   NaN
2  2020-01-03   7.6   6.7   8.3   NaN   NaN   NaN   8.4   NaN  1010.2   NaN
3  2020-01-04   8.2   6.7   

In [ ]:
#keeping only columns with at least 27 non-null values
df_temp = df_temp[['date','tavg','tmin','tmax','wspd','pres']]
#filling numerical series with mean of said series
for col in df_temp:
  if df_temp[col].dtype == 'float64':
    df_temp[col].fillna(np.mean(df_temp[col]),inplace=True)
print(df_temp.isnull().sum())

date    0
tavg    0
tmin    0
tmax    0
wspd    0
pres    0
dtype: int64


In [ ]:
# pd.options.display.max_info_rows = 4000000
df_taxi = pd.read_parquet((r'/content/drive/MyDrive/Project 2 intro to DL DFs/yellow_tripdata_2020-01.parquet')).sample(n=1000000)#.iloc[:200000,:]
df_taxi['store_and_fwd_flag'] = df_taxi['store_and_fwd_flag'].map({'Y': 1, 'N': 0})
print(df_taxi.info())
print(df_taxi.isnull().sum())
# print(df_taxi.head(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 1432517 to 4443048
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1000000 non-null  int64         
 1   tpep_pickup_datetime   1000000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  1000000 non-null  datetime64[ns]
 3   passenger_count        989787 non-null   float64       
 4   trip_distance          1000000 non-null  float64       
 5   RatecodeID             989787 non-null   float64       
 6   store_and_fwd_flag     989787 non-null   float64       
 7   PULocationID           1000000 non-null  int64         
 8   DOLocationID           1000000 non-null  int64         
 9   payment_type           1000000 non-null  int64         
 10  fare_amount            1000000 non-null  float64       
 11  extra                  1000000 non-null  float64       
 12  mta_tax               

In [ ]:
df_taxi.drop(columns='airport_fee',inplace=True)

# Merging both DFs

In [ ]:
df_taxi['date'] = df_taxi['tpep_pickup_datetime'].dt.date

In [ ]:
df = df_taxi.merge(df_temp,
                   how='left'
                   ,on='date')
df['trip_duration'] = pd.Series(map(lambda x: (x.seconds % 3600) // 60,df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])) 
# df.drop(columns=['date'],inplace=True)
df['pickup_hour'] = pd.Series(map(lambda x: str(x.strftime('%k')), df['tpep_pickup_datetime'])) 
# df['dropoff_hour'] = pd.Series(map(lambda x: str(x.strftime('%k')), df['tpep_dropoff_datetime'])) 
df['pickup_minute'] = pd.Series(map(lambda x: str(x.strftime('%M')), df['tpep_pickup_datetime'])) 
# df['dropoff_minute'] = pd.Series(map(lambda x: str(x.strftime('%M')), df['tpep_dropoff_datetime'])) 
df['day_of_month'] = pd.Series(map(lambda x: int(x.strftime('%e')) ,df['date']))
df['day_of_week'] = pd.Series(map(lambda x: int(x.strftime('%u')) ,df['date']))
del df_taxi,df_temp
df.drop(columns=['tpep_pickup_datetime','tpep_dropoff_datetime','date'],inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
print(df.head(5))
print(df.shape)

   VendorID  passenger_count  trip_distance  RatecodeID  store_and_fwd_flag  \
0         2              2.0           2.22         1.0                 0.0   
1         1              2.0           4.80         1.0                 0.0   
2         2              1.0           1.18         1.0                 0.0   
3         2              1.0           1.31         1.0                 0.0   
4         2              1.0           1.44         1.0                 0.0   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  ...  tavg  \
0           107           209             1         10.0    0.5  ...   2.6   
1           209           170             1         17.0    2.5  ...  -1.5   
2           141           163             1          7.0    0.0  ...  -2.2   
3           162           186             1          7.5    1.0  ...   6.5   
4           100           164             1         12.0    0.0  ...  -0.6   

   tmin  tmax  wspd    pres  trip_duration  pickup_hour 

In [ ]:
df[df['tavg'].isnull()]#.isnull().any()

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,tavg,tmin,tmax,wspd,pres,trip_duration,pickup_hour,pickup_minute,day_of_month,day_of_week


Not performing EDA since this was done during the first project, skipping straight to regression analysis part

In [ ]:
#splitting the dataset into 80/20 ratios
X = df.drop(columns=['trip_duration']).values
y = df['trip_duration'].values
# del df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
# print(X_train.shape)
# print(X_test.shape)

#adding another split to randomly reduce number of training rows to about 60 to speed up model training
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.80, random_state = 0)

print(X_train.shape)
print(X_test.shape)
# sys.exit('abc')
# del X,y

(791804, 25)
(197951, 25)


In [ ]:
#performing feature scaaling to 1/ ease computations and 2/ not having an independent variables dominating all the other independent variables
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#performing PCA to ease computations as well
from sklearn.decomposition import PCA
number_of_components = 15
pca = PCA(n_components = number_of_components)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_
print(f'With {number_of_components} principal components instead of {len(df.columns)-1} components I still obtain {round(sum(explained_variance)*100,2)} % of the variance')
# sys.exit('abc')

With 15 principal components instead of 25 components I still obtain 88.93 % of the variance


# Training the NNs

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Input
from sklearn.metrics import mean_squared_error
import plotly.offline as pyo
import plotly.graph_objects as go
from keras.callbacks import TensorBoard

## 1/ MLP

https://www.baeldung.com/cs/mlp-vs-dnn#:~:text=MLPs%20are%20neural%20networks%20with,to%20traditional%20machine%20learning%20algorithms.
according to this website, the main difference between a MLP and a DNN is the # of hidden layers. A MLP has fewer hidden layers than a DNN, among other things.

In [ ]:
model = Sequential()
model.add(Input(
    shape=(X_train.shape[1],)#(32,)
    ,name='inputLayer'
))
model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0)) #hidden layer # of nodes = avg # of nodes of the input and output layer
              , kernel_initializer = 'uniform' #initialize the weights according to a uniform distribution and close to 0
              , activation = 'relu' 
              , name = 'hiddenLayer1'
              )) 
model.add(Dense(units = 1
              , kernel_initializer = 'uniform' 
              , activation = 'linear' 
              , name = 'outputLayer'
              ))

In [ ]:
model.compile(
    optimizer='adam'
    ,loss='mse' #chose mse as a loss function because it can be proven that MSE is equal to the bias squared plus the variance, 
                #so minimizing this metric can reduce both bias and variance and therefore helps fighting back overfitting and underfitting
    ,metrics=['mae','mse']
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hiddenLayer1 (Dense)        (None, 8)                 128       
                                                                 
 outputLayer (Dense)         (None, 1)                 9         
                                                                 
Total params: 137
Trainable params: 137
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Fitting the MLP to the training set
tensorboard_callback = TensorBoard(
    log_dir = 'tboards/log/MLPAdam100Epochs'
    ,histogram_freq=1
    ,write_graph=True
    ,write_images=False
    ,update_freq='epoch'
)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)# ,min_delta=0.05          
model_history = model.fit(X_train,y_train
          ,batch_size=32#default value
          ,epochs=100
          ,validation_data = (X_test,y_test)
          ,callbacks=[tensorboard_callback,early_stopping]
          )

Epoch 1/100
19902/24744 [=======================>......] - ETA: 9s - loss: 42.4230 - mae: 4.2473 - mse: 42.4230

In [ ]:
try:
  model_history = pd.DataFrame(model_history.history)
  model_history['epoch'] = model_history.index+1
except:
  pass
trace0 = go.Scatter(x=model_history['epoch']
                    ,y=model_history['mse']
                    ,mode='lines+markers'
                    ,name='MLPTrain')
trace1 = go.Scatter(x=model_history['epoch']
                    ,y=model_history['val_mse']
                    ,mode='lines+markers'
                    ,name='MLPTest')
layout = go.Layout(title='MSE results through the epochs')
fig = go.Figure(data = [trace0,trace1]
                ,layout=layout)
fig.show()

## 2/ Linear regression

In [ ]:
model = Sequential()
model.add(Input(
    shape=(X_train.shape[1],)
    ,name='inputLayer'
))
model.add(Dense(units = 32#int(round((X_train.shape[1]+1)/2,0))
              , kernel_initializer = 'uniform'
              , activation = None
              , name = 'hiddenLayer1'
              ))
# model.add(Dense(units = 1#int(round((X_train.shape[1]+1)/2,0))
#               , kernel_initializer = 'uniform'
#               , activation = None
#               , name = 'hiddenLayer2'
#               ))
# model.add(Dense(
#     units=1
#     ,activation=None
#     ,name='outputLayer'
# ))

In [ ]:
model.compile(
    optimizer='adam'
    ,loss='mse'
    ,metrics=['mae','mse']
)

In [ ]:
model.summary()

In [ ]:
#Fitting the regression model to the training set
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)# ,min_delta=0.05          
model_history = model.fit(X_train,y_train
          ,batch_size=32#default value
          ,epochs=100
          ,callbacks=[early_stopping]
          ,validation_data = (X_test,y_test)
          )

In [ ]:
try:
  model_history = pd.DataFrame(model_history.history)
  model_history['epoch'] = model_history.index+1
except:
  pass
# model_history['epoch'] = model_history.epoch
trace2 = go.Scatter(x=model_history['epoch']
                    ,y=model_history['mse']
                    ,mode='lines+markers'
                    ,name='LRTrain')
trace3 = go.Scatter(x=model_history['epoch']
                    ,y=model_history['val_mse']
                    ,mode='lines+markers'
                    ,name='LRTest')
layout = go.Layout(title='MSE results through the epochs')
fig = go.Figure(data = [trace0,trace1,trace2,trace3]
                ,layout=layout)
fig.show()

## 3/ DNN

In [ ]:
#instantiating the model
model = Sequential()
#adding input and hidden layers
model.add(Input(
    shape=(X_train.shape[1],)#(32,)
    ,name='inputLayer'
))
model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0))
              , kernel_initializer = 'uniform'
              , activation = 'relu' 
              , name = 'hiddenLayer1'
              )) 
model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0))
              , kernel_initializer = 'uniform'
              , activation = 'relu' 
              , name = 'hiddenLayer2'
              )) 
model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0))
              , kernel_initializer = 'uniform'
              , activation = 'relu' 
              , name = 'hiddenLayer3'
              ))
model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0))
              , kernel_initializer = 'uniform'
              , activation = 'relu' 
              , name = 'hiddenLayer4'
              ))
#adding the output layer

model.add(Dense(units = 1
              , kernel_initializer = 'uniform' 
              , activation = 'linear' 
              , name = 'outputLayer'
              ))#softmax function: equivalent of sigmoid function for a dependent variable that has more than 2 categories (multi label classification)
# sys.exit('abc')

In [ ]:
#compiling the DNN
model.compile(
    optimizer='adam'#tf.keras.optimizers.Adam(learning_rate=1e-3)
    ,loss='mse'
    ,metrics=['mae','mse']
)
#perceptron model -> simple neural network with one neuron - and take a sigmoid activation function, you obtain a logistic regression function


In [ ]:
model.summary()

In [ ]:
#Fitting the DNN to the training set
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)# ,min_delta=0.05          
model_history= model.fit(X_train,y_train
          ,batch_size=32#default value
          ,epochs=100
          ,callbacks=[early_stopping]
          ,validation_data = (X_test,y_test)
          # ,patience=0
          )

In [ ]:
try:
  model_history = pd.DataFrame(model_history.history)
  model_history['epoch'] = model_history.index+1
except:
  pass
trace4 = go.Scatter(x=model_history['epoch']
                    ,y=model_history['mse']
                    ,mode='lines+markers'
                    ,name='DNNTrain')
trace5 = go.Scatter(x=model_history['epoch']
                    ,y=model_history['val_mse']
                    ,mode='lines+markers'
                    ,name='DNNTest')
layout = go.Layout(title='MSE results through the epochs'
                   ,xaxis=dict(title='Number of epochs')
                   ,yaxis=dict(title='MSE')
                   )
fig = go.Figure(data = [trace0,trace1,trace2,trace3,trace4,trace5]
                ,layout=layout)
fig.show()

In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_true=y_test
                         ,y_pred=y_pred)
print(mse)

# Hyperparameter tuning using GridSearch

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
def build_model(optimizer):#this functon just builds the architecture of our DNN
  model = Sequential()
  model.add(Input(
      shape=(X_train.shape[1],)#(32,)
      ,name='inputLayer'
  ))
  model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0))
                , kernel_initializer = 'uniform'
                , activation = 'relu' 
                , name = 'hiddenLayer1'
                )) 
  model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0))
                , kernel_initializer = 'uniform'
                , activation = 'relu' 
                , name = 'hiddenLayer2'
                )) 
  model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0))
                , kernel_initializer = 'uniform'
                , activation = 'relu' 
                , name = 'hiddenLayer3'
                ))
  model.add(Dense(units = int(round((X_train.shape[1]+1)/2,0))
                , kernel_initializer = 'uniform'
                , activation = 'relu' 
                , name = 'hiddenLayer4'
                ))
  model.add(Dense(units = 1
                , kernel_initializer = 'uniform' 
                , activation = 'linear' 
                , name = 'outputLayer'))
  model.compile(
      optimizer=optimizer
      ,loss='mse'
      ,metrics=['mae','mse']
  )
  model.summary()
  #Fitting the MLP to the training set
  return model
model = KerasClassifier(build_fn=build_model)
parameters = {'batch_size':[25,50]
              ,'epochs':[25,75]
              ,'optimizer':['adam','rmsprop','sgd']}
grid_search = GridSearchCV(estimator = model
                           ,param_grid = parameters
                           ,scoring = 'neg_mean_squared_error'
                           ,cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_mse = grid_search.best_score_
print(best_parameters)
print(best_mse)
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)         
# model_history = model.fit(X_train,y_train
#           # ,batch_size=32
#           # ,epochs=100
#           ,validation_data = (X_test,y_test)
#           ,callbacks=[early_stopping]
#           ) #gonna tune both parameters


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hiddenLayer1 (Dense)        (None, 8)                 128       
                                                                 
 hiddenLayer2 (Dense)        (None, 8)                 72        
                                                                 
 hiddenLayer3 (Dense)        (None, 8)                 72        
                                                                 
 hiddenLayer4 (Dense)        (None, 8)                 72        
                                                                 
 outputLayer (Dense)         (None, 1)                 9         
                                                                 
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning:

KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.



2851/2851 [==============================] - 6s 2ms/step - loss: 54.8256 - mae: 4.9109 - mse: 54.8256
Epoch 2/25
2851/2851 [==============================] - 5s 2ms/step - loss: 37.5299 - mae: 4.0993 - mse: 37.5299
Epoch 3/25
2851/2851 [==============================] - 7s 2ms/step - loss: 37.1760 - mae: 4.0802 - mse: 37.1760
Epoch 4/25
2851/2851 [==============================] - 5s 2ms/step - loss: 37.1495 - mae: 4.0719 - mse: 37.1495
Epoch 5/25
2851/2851 [==============================] - 5s 2ms/step - loss: 37.0009 - mae: 4.0636 - mse: 37.0009
Epoch 6/25
2851/2851 [==============================] - 5s 2ms/step - loss: 36.8933 - mae: 4.0651 - mse: 36.8933
Epoch 7/25
2851/2851 [==============================] - 5s 2ms/step - loss: 36.7210 - mae: 4.0585 - mse: 36.7210
Epoch 8/25
2851/2851 [==============================] - 5s 2ms/step - loss: 36.5256 - mae: 4.0490 - mse: 36.5256
Epoch 9/25
2851/2851 [==============================] - 5s 2ms/step - loss: 36.4001 - mae: 4.0333 - mse: 36